# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from config import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
# Read the CSV
csv_path = os.path.join("..", "weatherpy", "output_data", "api-data.csv")
weather_df = pd.read_csv(csv_path)

# Display
weather_df

,Unnamed: 0,City,Lat,Lng,Date and Time,Max Temperature,Humidity,Cloudiness,Windspeed
0,0,Kapaa,22.08,-159.32,2020-10-20 16:55:17,73.00,79,46,8.55
1,1,Cheuskiny,22.08,-159.32,2020-10-20 16:55:17,73.00,79,46,8.55
2,2,Belmonte,-15.86,-38.88,2020-10-20 16:55:26,77.00,83,90,6.93
3,3,Port Elizabeth,-33.92,25.57,2020-10-20 16:53:30,62.60,72,10,19.46
4,4,Norman Wells,65.28,-126.83,2020-10-20 16:55:32,17.60,92,75,9.17
...,...,...,...,...,...,...,...,...,...
574,574,Samusu,-16.23,39.91,2020-10-20 17:08:12,78.96,72,0,10.94
575,575,Jubayl,-16.23,39.91,2020-10-20 17:08:12,78.96,72,0,10.94
576,576,Leshukonskoye,64.90,45.76,2020-10-20 17:01:03,25.41,98,88,6.13
577,577,Bonavista,48.65,-53.11,2020-10-20 17:03:17,55.24,81,99,14.27


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
# Set empty lists to capture coordinates and humidities
coordinates = []
humidities = []

# Iterate through weather_df and append coordinates and humidities to empty lists
for index, row in weather_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    target_coordinates = [lat, lng]
    coordinates.append(target_coordinates)
    humid = row["Humidity"]
    humidities.append(humid)

In [41]:
# Create a Heatmap with GMAPS
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidities, dissipating=False, max_intensity=100, point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

  * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [45]:
# Find ideal weather conditions for vacation spots and display

vacation_df = weather_df.loc[(weather_df["Max Temperature"] < 80) &  (weather_df["Max Temperature"] > 70) & (weather_df["Windspeed"] < 10)]
vacation_df

,Unnamed: 0,City,Lat,Lng,Date and Time,Max Temperature,Humidity,Cloudiness,Windspeed
0,0,Kapaa,22.08,-159.32,2020-10-20 16:55:17,73.00,79,46,8.55
1,1,Cheuskiny,22.08,-159.32,2020-10-20 16:55:17,73.00,79,46,8.55
2,2,Belmonte,-15.86,-38.88,2020-10-20 16:55:26,77.00,83,90,6.93
30,30,Mbandaka,0.05,18.26,2020-10-20 16:56:03,77.34,79,23,1.66
50,50,Virginia Beach,36.85,-75.98,2020-10-20 16:55:52,75.20,73,75,3.74
...,...,...,...,...,...,...,...,...,...
558,558,Chimoio,-19.12,33.48,2020-10-20 17:07:54,79.12,40,0,8.59
559,559,Lichinga,-13.31,35.24,2020-10-20 17:07:55,71.38,51,63,7.40
560,560,Misratah,32.38,15.09,2020-10-20 17:07:56,72.25,55,5,8.39
562,562,Karachi,24.91,67.08,2020-10-20 17:03:30,78.80,44,0,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [117]:
# Create empty column in weather_df
vacation_df["Hotel"] = ""

# Clean up dataframe

# hotel_df = vacation_df.drop(columns={"Unnamed: 0"})
hotel_df = vacation_df
hotel_df = hotel_df.reset_index()
hotel_df = hotel_df.drop(columns="index")
hotel_df

,City,Lat,Lng,Date and Time,Max Temperature,Humidity,Cloudiness,Windspeed,Hotel
0,Kapaa,22.08,-159.32,2020-10-20 16:55:17,73.00,79,46,8.55,
1,Cheuskiny,22.08,-159.32,2020-10-20 16:55:17,73.00,79,46,8.55,
2,Belmonte,-15.86,-38.88,2020-10-20 16:55:26,77.00,83,90,6.93,
3,Mbandaka,0.05,18.26,2020-10-20 16:56:03,77.34,79,23,1.66,
4,Virginia Beach,36.85,-75.98,2020-10-20 16:55:52,75.20,73,75,3.74,
...,...,...,...,...,...,...,...,...,...
91,Chimoio,-19.12,33.48,2020-10-20 17:07:54,79.12,40,0,8.59,
92,Lichinga,-13.31,35.24,2020-10-20 17:07:55,71.38,51,63,7.40,
93,Misratah,32.38,15.09,2020-10-20 17:07:56,72.25,55,5,8.39,
94,Karachi,24.91,67.08,2020-10-20 17:03:30,78.80,44,0,4.70,


In [120]:
# Set the base URL for API calls
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set some parameters outside the for loop
target_search = "hotels"
target_radius = 5000
target_type = "lodging"

# iterate through hotels_df and add nearest hotel

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # Update parameters for each call with the new lat and lng, import previously set variables
    params = {
        "location": f"{lat}, {lng}", 
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }
    
    # Try, as not all requests bring back results
    try:
        response = requests.get(base_url, params=params).json()
        hotel_name = response["results"][0]["name"]
    # Print out the failed requests
    except IndexError:
        print(f"Hotel not found for {row['City']}")
    
    # Append to new dataframe column
    hotel_df.loc[index, "Hotel"] = hotel_name

Hotel not found for Kopargaon
Hotel not found for Twante
Hotel not found for Lakshettipet
Hotel not found for Kamenskoye
Hotel not found for Najran
Hotel not found for Timizart
Hotel not found for Demba
Hotel not found for Galesong
Hotel not found for Kieta
Hotel not found for Tautira
Hotel not found for Awjilah
Hotel not found for Labutta
Hotel not found for Lodja
Hotel not found for Beloha
Hotel not found for Ayna
Hotel not found for Camacupa
Hotel not found for Dingle
Hotel not found for Kasongo-Lunda
Hotel not found for Rafai


In [121]:
# Display with Hotel Names
hotel_df

,City,Lat,Lng,Date and Time,Max Temperature,Humidity,Cloudiness,Windspeed,Hotel
0,Kapaa,22.08,-159.32,2020-10-20 16:55:17,73.00,79,46,8.55,Hilton Garden Inn Kauai Wailua Bay
1,Cheuskiny,22.08,-159.32,2020-10-20 16:55:17,73.00,79,46,8.55,Secret Falls Retreat
2,Belmonte,-15.86,-38.88,2020-10-20 16:55:26,77.00,83,90,6.93,Pousada Belmonte
3,Mbandaka,0.05,18.26,2020-10-20 16:56:03,77.34,79,23,1.66,Nina River Hotel
4,Virginia Beach,36.85,-75.98,2020-10-20 16:55:52,75.20,73,75,3.74,Hilton Virginia Beach Oceanfront
...,...,...,...,...,...,...,...,...,...
91,Chimoio,-19.12,33.48,2020-10-20 17:07:54,79.12,40,0,8.59,Hotel Amirana
92,Lichinga,-13.31,35.24,2020-10-20 17:07:55,71.38,51,63,7.40,Girassol Lichinga
93,Misratah,32.38,15.09,2020-10-20 17:07:56,72.25,55,5,8.39,فندق الماسة
94,Karachi,24.91,67.08,2020-10-20 17:03:30,78.80,44,0,4.70,Elegant Palace Guest House Karachi


In [124]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [125]:
# # NOTE: Do not change any of the code in this cell

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]

In [127]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))